# Transfer Learning par extraction de features dans un CNN

In [1]:
import argparse
import os
import time

import torch.backends.cudnn as cudnn
import torch.nn.parallel
import torch.utils.data
import torchvision.models as models
from PIL import Image
from torch.nn import functional as F
import torchvision
import torch.nn as nn 
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
import pickle, PIL
from sklearn.svm import LinearSVC

In [2]:
torchvision.models.vgg.model_urls["vgg16"] = "http://webia.lip6.fr/~robert/cours/rdfia/vgg16-397923af.pth"
os.environ["TORCH_MODEL_ZOO"] = "/tmp/torch"
PRINT_INTERVAL = 50
CUDA = False
vgg16 = torchvision.models.vgg16(pretrained=True)

/anaconda3/lib/python3.6/site-packages/torch/hub.py:463: UserWarning: TORCH_MODEL_ZOO is deprecated, please use env TORCH_HOME instead
  warnings.warn('TORCH_MODEL_ZOO is deprecated, please use env TORCH_HOME instead')


In [3]:
def get_dataset(batch_size, path):
    mean = [0.485, 0.456, 0.406]
    std = [0.0229,0.224,0.225]
    # Cette fonction permet de recopier 3 fois une image qui
    # ne serait que sur 1 channel (donc image niveau de gris)
    # pour la "transformer" en image RGB. Utilisez la avec
    # transform.Lambda
    def duplicateChannel(img):
        img = img.convert('L')
        np_img = np.array(img, dtype=np.uint8)
        np_img = np.dstack([np_img, np_img, np_img])
        img = Image.fromarray(np_img, 'RGB')
        return img

    train_dataset = datasets.ImageFolder(path+'/train',
        transform=transforms.Compose([ # TODO Pré-traitement à faire
            transforms.Lambda(lambda img : duplicateChannel(img)),
            transforms.Lambda(lambda img : img.resize((224, 224), PIL.Image.BILINEAR)),
            transforms.ToTensor(),
            transforms.Normalize(mean,std)
        ]))
    val_dataset = datasets.ImageFolder(path+'/test',
        transform=transforms.Compose([ # TODO Pré-traitement à faire
            transforms.Lambda(lambda img : duplicateChannel(img)),
            transforms.Lambda(lambda img : img.resize((224, 224), PIL.Image.BILINEAR)),
            transforms.ToTensor(),
            transforms.Normalize(mean,std)
        ]))

    train_loader = torch.utils.data.DataLoader(train_dataset,
                        batch_size=batch_size, shuffle=False, pin_memory=CUDA, num_workers=2)
    val_loader = torch.utils.data.DataLoader(val_dataset,
                        batch_size=batch_size, shuffle=False, pin_memory=CUDA, num_workers=2)

    return train_loader, val_loader


In [4]:
import os, torchvision
torchvision.models.vgg.model_urls["vgg16"] ="http://webia.lip6.fr/~robert/cours/rdfia/vgg16-397923af.pth"
os.environ["TORCH_HOME"] = "/tmp/torch"
vgg16 = torchvision.models.vgg16(pretrained=True)

/anaconda3/lib/python3.6/site-packages/torch/hub.py:463: UserWarning: TORCH_MODEL_ZOO is deprecated, please use env TORCH_HOME instead
  warnings.warn('TORCH_MODEL_ZOO is deprecated, please use env TORCH_HOME instead')


In [5]:
def extract_features(data, model):
    # TODO init features matrices
    X = np.zeros((len(data),data.batch_size,4096)) # 4096 dimension de l'avant derniere couche de VGG16.
    y = np.zeros((len(data),data.batch_size))
    model.eval()
    with torch.no_grad():
        for i, (x, target) in enumerate(data):
            if i % PRINT_INTERVAL == 0:
                print('Batch {0:03d}/{1:03d}'.format(i, len(data)))
            if CUDA:
                x = x.cuda()
            
            features = model(x)
            X[i] = features.detach().numpy()
            y[i]= target.detach().numpy()
            
            
    return X, y


In [6]:

def reshape_no_batch(X,y):
    # Enleve les batch et reshape les données numpy. 
    X = X.reshape((X.shape[0]*X.shape[1],X.shape[2]))
    y = y.reshape((y.shape[0]*y.shape[1]))

    return X,y


In [11]:
class VGG16relu7(nn.Module):
    def __init__(self):
        super(VGG16relu7, self).__init__()
        # recopier toute la partie convolutionnelle
        self.features = nn.Sequential( *list(vgg16.features.children()))
        # garder une partie du classifieur, -2 pour s'arrêter à relu7
        self.classifier = nn.Sequential(*list(vgg16.classifier.children())[:-3])
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [12]:
def main():
    path = '15SceneData' 
    batch_size = 4 
    
    print('Instanciation de VGG16')
    vgg16 = models.vgg16(pretrained=True)

    print('Instanciation de VGG16relu7')
    model = VGG16relu7() # TODO À remplacer par un reseau tronché pour faire de la feature extraction

    model.eval()
    if CUDA: # si on fait du GPU, passage en CUDA
        model = model.cuda()

    # On récupère les données
    print('Récupération des données')
    train, test = get_dataset(batch_size, path)

    
    # Extraction des features
    print('Feature extraction')
    if not os.path.exists('numpy_object/X_train2.npy') : 
        X_train, y_train = extract_features(train, model)
        np.save('numpy_object/X_train2.npy', X_train)
        np.save('numpy_object/y_train2.npy', y_train)
    else:
        print("Chargement des fichiers X_train2.npy et y_train2.npy")
        X_train = np.load('numpy_object/X_train2.npy')
        y_train = np.load('numpy_object/y_train2.npy')
        
    if not os.path.exists('numpy_object/X_test2.npy') : 
        X_test, y_test = extract_features(test, model)
        np.save('numpy_object/X_test2.npy', X_test)
        np.save('numpy_object/y_test2.npy', y_test)
    else:
        X_test = np.load('numpy_object/X_test2.npy')
        y_test = np.load('numpy_object/y_test2.npy')


    X_train,y_train = reshape_no_batch(X_train,y_train)
    X_test,y_test = reshape_no_batch(X_test,y_test)

    
    
    print('Apprentissage des SVM')
    svm = LinearSVC(C=1.0)
    svm.fit(X_train,y_train)
    accuracy = svm.score(X_test,y_test)
    print("SVM accuraccy :",accuracy)

    input("done")
    return X_train, y_train, X_test, y_test


In [13]:
CUDA = False 
X_train, y_train, X_test, y_test = main()

Instanciation de VGG16


/anaconda3/lib/python3.6/site-packages/torch/hub.py:463: UserWarning: TORCH_MODEL_ZOO is deprecated, please use env TORCH_HOME instead
  warnings.warn('TORCH_MODEL_ZOO is deprecated, please use env TORCH_HOME instead')


Instanciation de VGG16relu7
Récupération des données
Feature extraction
Batch 000/375
Batch 050/375
Batch 100/375
Batch 150/375
Batch 200/375
Batch 250/375
Batch 300/375
Batch 350/375
Batch 000/747
Batch 050/747
Batch 100/747
Batch 150/747
Batch 200/747
Batch 250/747
Batch 300/747
Batch 350/747
Batch 400/747
Batch 450/747
Batch 500/747
Batch 550/747
Batch 600/747
Batch 650/747
Batch 700/747
Apprentissage des SVM
SVM accuraccy : 0.7623828647925034
done
